In [1]:
import numpy as np 
import pandas as pd
from functools import reduce

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

In [2]:
dfevent = pd.read_csv(r"~\Downloads\telstra_recruiting_network\event_type.csv")
dflog = pd.read_csv(r"~\Downloads\telstra_recruiting_network\log_feature.csv")
dfresource = pd.read_csv(r"~\Downloads\telstra_recruiting_network\resource_type.csv")
dfsubmission = pd.read_csv(r"~\Downloads\telstra_recruiting_network\sample_submission.csv")
dftest = pd.read_csv(r"~\Downloads\telstra_recruiting_network\test.csv")
dftrain = pd.read_csv(r"~\Downloads\telstra_recruiting_network\train.csv")

In [142]:
dflog = pd.read_csv(r"~\Downloads\telstra_recruiting_network\log_feature.csv")

In [3]:
df = [dftrain,dfevent, dflog, dfresource]

In [4]:
dftrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7381 entries, 0 to 7380
Data columns (total 3 columns):
id                7381 non-null int64
location          7381 non-null object
fault_severity    7381 non-null int64
dtypes: int64(2), object(1)
memory usage: 173.1+ KB


In [5]:
df1 = reduce(lambda left,right: pd.merge(left,right,on='id',how='left'), df)

In [6]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61839 entries, 0 to 61838
Data columns (total 7 columns):
id                61839 non-null int64
location          61839 non-null object
fault_severity    61839 non-null int64
event_type        61839 non-null object
log_feature       61839 non-null object
volume            61839 non-null int64
resource_type     61839 non-null object
dtypes: int64(3), object(4)
memory usage: 3.8+ MB


Aquí se observa que los ids se repoten hasta 11 veces por difentes event_types.

In [7]:
dfevent.id = dfevent.id.astype('category')
dg = dfevent.groupby('id').aggregate('count')
dg.event_type=dg.event_type.astype('category')
dg.reset_index(inplace=True)
dg.groupby('event_type').aggregate('count')

Lo que se hara acontinuación es ver que event_types son más relevantes dado su repitición. Hay que recordar que existen en la base de dfevent: 31170 registros. Y nosotros simplicidad agarraremos los event_type que más se repiten por el dilema de pareto. 
event_type: 11,35,34,15 y 20. Conforman alrededor del 84% de los datos. Por relevancia solo incluiremos esos unicamente.

In [40]:
dfevent.groupby('event_type').aggregate('count').sort_values(by=['id'],ascending=False).head()

,id
event_type,
event_type 11,7888
event_type 35,6615
event_type 34,5927
event_type 15,4395
event_type 20,1458


In [60]:
dfevent.id = dfevent.id.astype('int64')
principal=['event_type 11','event_type 35','event_type 34','event_type 15','event_type 20']
dfevent=dfevent.loc[dfevent.event_type.isin(principal_event),:]

Observamos que efectivamente si un "id" llega a tener cinco diferentes tipos de fuentes.

In [69]:
dfresource.id = dfresource.id.astype('category')
dg = dfresource.groupby('id').aggregate('count')
dg.resource_type=dg.resource_type.astype('category')
dg.reset_index(inplace=True)
dg.groupby('resource_type').aggregate('count')

,id
resource_type,
1,16421
2,1814
3,257
4,44
5,16


In [70]:
dfresource.groupby('resource_type').aggregate('count').sort_values(by=['id'],ascending=False).head()

,id
resource_type,
resource_type 8,10268
resource_type 2,8918
resource_type 6,582
resource_type 7,498
resource_type 4,330


In [71]:
dfresource.id = dfresource.id.astype('int64')
principal=['resource_type 8','resource_type 2']
dfresource=dfresource.loc[dfresource.resource_type.isin(principal),:]

In [75]:
dflog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58671 entries, 0 to 58670
Data columns (total 3 columns):
id             58671 non-null int64
log_feature    58671 non-null object
volume         58671 non-null int64
dtypes: int64(2), object(1)
memory usage: 1.3+ MB


In [157]:
dflog.groupby(['log_feature', 'volume']).size().head()

log_feature  volume
feature 134  1         1225
             2          128
             3           23
             4           11
             5            4
dtype: int64

Averiguaremos si el volumen corresponde a un valor unico de log_feature

58671*0.8=46936.8

In [131]:
dflog.log_feature.value_counts().head()

feature 312    5267
feature 232    4754
feature 82     3472
feature 203    2823
feature 313    2145
Name: log_feature, dtype: int64

In [168]:
#dflog.loc[(dflog.log_feature=='feature 313'),:].groupby(['log_feature', 'volume']).size()
dflog.volume.value_counts().head() #Nos quedaremos hasta el valor 5

1    11244
2     5575
3     2815
4     2380
6     1536
Name: volume, dtype: int64

In [144]:
dflog.id = dflog.id.astype('int64')
principal=['feature 312','feature 232','feature 82','feature 203','feature 313','feature 233','feature 307','feature 54'
          ,'feature 170','feature 71','feature 315','feature 134','feature 80','feature 235','feature 193','feature 219','feature 68']
dflog=dflog.loc[dflog.log_feature.isin(principal),:]
dflog=dflog.loc[dflog.volume<7,:]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25070 entries, 0 to 58667
Data columns (total 3 columns):
id             25070 non-null int64
log_feature    25070 non-null object
volume         25070 non-null int64
dtypes: int64(2), object(1)
memory usage: 783.4+ KB


In [ ]:
df1.fault_severity = df1['fault_severity'].astype('category')
df1.volume = df1['volume'].astype('int32')
df1.location = df1['location'].astype('category')
df1.event_type = df1['event_type'].astype('category')
df1.log_feature = df1['log_feature'].astype('category')
df1.resource_type = df1['resource_type'].astype('category')

In [ ]:
sns.pairplot(data=df1.drop(['id'],axis=1), hue="fault_severity")

In [ ]:
df1.event_type.unique 
df1.log_feature.unique  
df1.resource_type.unique
df1.location.unique

In [ ]:
x_category = ['location','event_type','log_feature','resource_type']
x_numerical = ['volume']
target = ['fault_severity']

In [ ]:
dfX = df1[x_category]

In [ ]:
le = preprocessing.LabelEncoder()
for i in range(len(dfX.columns)):
    dfX.iloc[:,i]=le.fit_transform(dfX.iloc[:,i])

In [ ]:
df1.head()